<img src="https://cybersecurity-excellence-awards.com/wp-content/uploads/2017/06/366812.png">


<h1><center>Darwin Unsupervised Model Building </center></h1>


# Prior to getting started:

First, if you have just received a new api key from support, you will need to register your key and create a new user (see Register user)

Second, in the Environment Variables cell: 
1. Set your username and password to ensure that you're able to log in successfully
2. Set the path to the location of your datasets if you are using your own data.  The path is set for the examples.
  <br><b>NOTE:</b> We provide two ways to analyze feature importance. One is to use the entire dataset; the other one is to analyze a few samples to understand individual samples. In the latter case, we advise users to use a small dataset (<=500) because it takes long time to process individual samples. 

Here are a few things to be mindful of:
1. For every run, check the job status (i.e. requested, failed, running, completed) and wait for job to complete before proceeding. 
2. If you're not satisfied with your model and think that Darwin can benefit from extra training, use the resume function.

## Set Darwin SDK

In [1]:
from amb_sdk.sdk import DarwinSdk
ds = DarwinSdk()
ds.set_url('https://amb-demo-api.sparkcognition.com/v1/')

(True, 'https://amb-demo-api.sparkcognition.com/v1/')

## Environment Variables

In [2]:
#Set your user id and password accordingly
USER='idunlap@rocketmail.com'
PW='5uVGHsTHrQ'

# Set path to datasets - The default below assumes Jupyter was started from amb-sdk/examples/Enterprise/
# Modify accordingly if you wish to use your own data
PATH_TO_DATASET = 'sets/'
TRAIN_DATASET='17_18_Season_Stats_Per_100_Possessions_SPP.csv'
PREDICT_DATASET = '16_17_Season_Stats_Per_100_Possessions_SPP.csv'

# A timestamp is used to create a unique name in the event you execute the workflow multiple times or with 
# different datasets.  File names must be unique in Darwin.
import datetime
ts = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())

## Import necessary libraries

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
from time import sleep
import os

# User Login

In [4]:
status, msg = ds.auth_login_user(USER,PW)
if not status:
    print(msg)
else:
    print('You are logged in.')

You are logged in.


# Data Upload

**Read dataset and view a file snippet**

In [5]:
# Preview dataset
df = pd.read_csv(os.path.join(PATH_TO_DATASET, TRAIN_DATASET))
df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,TRB,AST,STL,BLK,TOV,PF,PTS,string_field_29,ORtg,DRtg
0,8,Jarrett Allen\allenja01,C,19,BRK,72,31,1441,7.9,13.4,...,13.1,1.7,0.9,3.0,2.8,5.0,19.8,NaN,119.0,108
1,15,Ike Anigbogu\anigbik01,C,19,IND,11,0,30,6.7,15.0,...,15.0,0.0,1.7,5.0,3.3,1.7,21.7,NaN,114.0,104
2,4,Bam Adebayo\adebaba01,C,20,MIA,69,19,1368,6.4,12.5,...,14.0,3.7,1.2,1.5,2.4,5.1,17.5,NaN,116.0,105
3,62,Tony Bradley\bradlto01,C,20,UTA,9,0,29,5.2,19.0,...,19.0,1.7,0.0,0.0,0.0,1.7,13.8,NaN,95.0,107
4,75,Thomas Bryant\bryanth01,C,20,LAL,15,0,72,5.3,14.0,...,11.3,4.0,0.7,1.3,1.3,4.0,14.6,NaN,100.0,108


**Upload dataset to Darwin**

In [6]:
# Upload dataset
status, dataset = ds.upload_dataset(os.path.join(PATH_TO_DATASET, TRAIN_DATASET))
print(status)
print(dataset)

if not status:
    print(dataset)

False
400: BAD REQUEST - {"message": "Dataset already exists"}

400: BAD REQUEST - {"message": "Dataset already exists"}



# Analyze Data
Before creating a model, users need to analyze data and clean data first. 

In [7]:
status, analyze_id = ds.analyze_data(TRAIN_DATASET, 
                                     job_name = 'Darwin_analyze_data_job' + "-" + ts, 
                                     artifact_name = 'Darwin_analyze_data_artifact' + "-" + ts)
sleep(1)
if status:
    ds.wait_for_job('Darwin_analyze_data_job' + "-" + ts)
else:
    print(analyze_id)

{'status': 'Requested', 'starttime': '2019-04-17T22:23:37.322187', 'endtime': None, 'percent_complete': 0, 'job_type': 'AnalyzeData', 'loss': None, 'generations': None, 'dataset_names': ['17_18_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': ['Darwin_analyze_data_artifact-20190417222338'], 'model_name': None, 'job_error': None}
{'status': 'Running', 'starttime': '2019-04-17T22:23:37.322187', 'endtime': None, 'percent_complete': 10, 'job_type': 'AnalyzeData', 'loss': None, 'generations': None, 'dataset_names': ['17_18_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': ['Darwin_analyze_data_artifact-20190417222338'], 'model_name': None, 'job_error': None}
{'status': 'Running', 'starttime': '2019-04-17T22:23:37.322187', 'endtime': None, 'percent_complete': 10, 'job_type': 'AnalyzeData', 'loss': None, 'generations': None, 'dataset_names': ['17_18_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': ['Darwin_analyze_data_artifact-20190417222338'], 'model_na

In [8]:
ds.lookup_job_status_name(analyze_id['job_name'])

(True,
 {'status': 'Complete',
  'starttime': '2019-04-17T22:23:37.322187',
  'endtime': '2019-04-17T22:24:39.067',
  'percent_complete': 100,
  'job_type': 'AnalyzeData',
  'loss': None,
  'generations': None,
  'dataset_names': ['17_18_Season_Stats_Per_100_Possessions_SPP.csv'],
  'artifact_names': ['Darwin_analyze_data_artifact-20190417222338'],
  'model_name': None,
  'job_error': ''})

# Clean Data

Starting Version 1.6, Darwin SDK offers a way to clean your data outside of model training. Every dataset needs to be cleaned before creating a model. There is no need to save the cleaned data and upload it. 

In [9]:
# Clean dataset
status, job_id = ds.clean_data(dataset_name=TRAIN_DATASET)
if not status:
    print(job_id)
else:
    print('Data has been successfully cleaned!')

Data has been successfully cleaned!


# Create and Train Model 

To build unsupervised models, which cluster data and perform anomaly detection, Darwin goes through the following steps:
1. Determines an approximate number of clusters to start with using a single pass with a hierarchical method
2. Iterates on subsets of the data using a Spectral-Net algorithm to determine the ideal number of clusters
3. Proceeds to cluster the data using a Spectral-Net approach

In the cell below, specify the parameters used to create the model:
- model: the name of your model
- max_epochs: the number of epochs to train the model, one epoch indicates one scan of the entire dataset
- n_clusters: the number of clusters, either an integer or 'auto', if left with 'auto', the unsupervised algorithm will compute a number for you

In [10]:
# Build model
model = "model" + "-" + ts
max_epochs = 20
n_clusters = 3
#n_clusters = 'auto'
status, job_id = ds.create_model(dataset_names=TRAIN_DATASET,
                                 model_name=model,
                                 max_epochs=max_epochs,
                                 n_clusters=n_clusters)
sleep(1)
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Running', 'starttime': '2019-04-17T22:24:56.651785', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['17_18_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': None, 'model_name': 'model-20190417222338', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-17T22:24:56.651785', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['17_18_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': None, 'model_name': 'model-20190417222338', 'job_error': ''}
{'status': 'Complete', 'starttime': '2019-04-17T22:24:56.651785', 'endtime': '2019-04-17T22:25:26.674666', 'percent_complete': 100, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['17_18_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': None, 'model_name': 'model-20190417222338', 'job_error': ''}


In [11]:
# look up job status
ds.lookup_job_status_name(job_id['job_name'])

(True,
 {'status': 'Complete',
  'starttime': '2019-04-17T22:24:56.651785',
  'endtime': '2019-04-17T22:25:26.674666',
  'percent_complete': 100,
  'job_type': 'TrainModel',
  'loss': None,
  'generations': 0,
  'dataset_names': ['17_18_Season_Stats_Per_100_Possessions_SPP.csv'],
  'artifact_names': None,
  'model_name': 'model-20190417222338',
  'job_error': ''})

In [12]:
# look up the model
ds.lookup_model_name(job_id['model_name'])

(True,
 {'type': 'Unsupervised',
  'updated_at': '2019-04-17T22:25:26.667318',
  'trained_on': ['17_18_Season_Stats_Per_100_Possessions_SPP.csv'],
  'loss': None,
  'generations': 0,
  'parameters': {'n_clusters': 3,
   'max_generation': 20,
   'train_time': '00:10',
   'recurrent': None,
   'max_unique_values': 50,
   'max_int_uniques': 15,
   'impute': 'mean',
   'big_data': False},
  'description': {'model': "UnsupervisedPipeline(anomaly=False, anomaly_prior=0.0015, auto_save_per=10,\n           clustering=True, clustermethod='GaussianMixture',\n           job_id='8a1484de-6189-11e9-882d-77778aae508c',\n           max_generation=20, max_time=600,\n           model_file='models/8f7f1eea-4fc6-11e9-ba76-eb31f920f59e_model-20190417222338',\n           n_clusters=3, preproc_anomaly=None, recurrent=None, verbose=2)",
   'genome_type': 'Unsupervised'},
  'train_time_seconds': 30,
  'algorithm': None,
  'running_job_id': None})

## Extra Training (Optional)
Run the following cell for extra training, no need to specify parameters

In [13]:
# Train some more
extra_epochs = 10
status, job_id = ds.resume_training_model(dataset_names=TRAIN_DATASET,
                                          model_name=model,
                                          max_epochs=extra_epochs,
                                          n_clusters=n_clusters)
sleep(1)
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Running', 'starttime': '2019-04-17T22:25:29.046222', 'endtime': None, 'percent_complete': 0, 'job_type': 'UpdateModel', 'loss': None, 'generations': 0, 'dataset_names': ['17_18_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': None, 'model_name': 'model-20190417222338', 'job_error': ''}
{'status': 'Complete', 'starttime': '2019-04-17T22:25:29.046222', 'endtime': '2019-04-17T22:25:36.777978', 'percent_complete': 100, 'job_type': 'UpdateModel', 'loss': None, 'generations': 0, 'dataset_names': ['17_18_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': None, 'model_name': 'model-20190417222338', 'job_error': ''}


## Predict
Run the following cell for prediction

In [14]:
# Test model
status, artifact = ds.run_model(TRAIN_DATASET, 
                                model, 
                                supervised=False)
sleep(1)
ds.wait_for_job(artifact['job_name'])

{'status': 'Running', 'starttime': '2019-04-17T22:25:46.251578', 'endtime': None, 'percent_complete': 0, 'job_type': 'RunModel', 'loss': None, 'generations': 0, 'dataset_names': ['17_18_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': ['ad0a2a12abb14d21918a913758745a05'], 'model_name': 'model-20190417222338', 'job_error': ''}
{'status': 'Complete', 'starttime': '2019-04-17T22:25:46.251578', 'endtime': '2019-04-17T22:25:51.82342', 'percent_complete': 100, 'job_type': 'RunModel', 'loss': None, 'generations': 0, 'dataset_names': ['17_18_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': ['ad0a2a12abb14d21918a913758745a05'], 'model_name': 'model-20190417222338', 'job_error': ''}


(True, 'Job completed')

In [15]:
# Get predictions
status, pred_file = ds.download_artifact(artifact['artifact_name'])

In [16]:
# View prediction
df = pd.read_csv(pred_file['filename'])
df.head()

,anomaly_score,predict_proba,prediction
0,-8.491837,"[0.0, 2.7237552576732322e-14, 0.9999999999999729]",2
1,-8.491837,"[0.0, 1.8755272952062358e-14, 0.9999999999999813]",2
2,-8.491837,"[0.0, 1.9748093129848906e-13, 0.9999999999998028]",2
3,-8.491837,"[0.0, 1.3463266156192075e-12, 0.9999999999986537]",2
4,-8.491837,"[0.0, 4.581518296236828e-11, 0.9999999999541849]",2


## Analyze Model
Analyze model provides feature importance ranked by the model. It indicates a general view of which features pose a bigger impact on the model

In [17]:
status, analyze_id = ds.analyze_model(job_id['model_name'], 
                                      job_name='Darwin_analyze_model_job-' + ts, 
                                      artifact_name='Darwin_analyze_model_artifact-' + ts)
sleep(1)
if status:
    ds.wait_for_job('Darwin_analyze_model_job-' + ts)
else:
    print(analyze_id)

{'status': 'Running', 'starttime': '2019-04-17T22:26:03.767726', 'endtime': None, 'percent_complete': 0, 'job_type': 'AnalyzeModel', 'loss': None, 'generations': 0, 'dataset_names': None, 'artifact_names': ['Darwin_analyze_model_artifact-20190417222338'], 'model_name': 'model-20190417222338', 'job_error': ''}
{'status': 'Complete', 'starttime': '2019-04-17T22:26:03.767726', 'endtime': '2019-04-17T22:26:09.134701', 'percent_complete': 100, 'job_type': 'AnalyzeModel', 'loss': None, 'generations': 0, 'dataset_names': None, 'artifact_names': ['Darwin_analyze_model_artifact-20190417222338'], 'model_name': 'model-20190417222338', 'job_error': ''}


In [18]:
ds.lookup_job_status_name('Darwin_analyze_model_job-' + ts)

(True,
 {'status': 'Complete',
  'starttime': '2019-04-17T22:26:03.767726',
  'endtime': '2019-04-17T22:26:09.134701',
  'percent_complete': 100,
  'job_type': 'AnalyzeModel',
  'loss': None,
  'generations': 0,
  'dataset_names': None,
  'artifact_names': ['Darwin_analyze_model_artifact-20190417222338'],
  'model_name': 'model-20190417222338',
  'job_error': ''})

Downloade and print the top 10 features

In [19]:
status, feature_importance = ds.download_artifact('Darwin_analyze_model_artifact-' + ts)
feature_importance[:10]

Pos = PG    0.158379
TRB         0.057819
AST         0.050054
BLK         0.049313
_3P         0.048095
FG_         0.044084
ORB         0.039314
Pos = PF    0.039167
Age         0.033771
Pos = SG    0.032652
dtype: float64

## Analyze Prediction
Different from Analyze Model, the Analyze Prediction provides a way to analyze feature importance for each data point. The output estimates how each feature added or subtracted from a known base-value to result in the overall prediction that was made.  <br>
**You need to set the path to the dataset which contains all the samples you want to analyze (max rows = 500)**

In [20]:
# Upload the data that you are interested in feature importance (max: 500 rows)
status, dataset = ds.upload_dataset(os.path.join(PATH_TO_DATASET, PREDICT_DATASET))
if not status:
    print(dataset)
    
if status:
    dataset_by_row=dataset['dataset_name']
else:
    print("Upload data failed!")

400: BAD REQUEST - {"message": "Dataset already exists"}

Upload data failed!


In [21]:
status, analyze_id = ds.analyze_predictions(job_id['model_name'], 
                                            PREDICT_DATASET, 
                                            job_name='Analyze_prediction_job-' + ts, 
                                            artifact_name='Analyze_prediction_artifact-' + ts)
sleep(1)
if status:
    ds.wait_for_job('Analyze_prediction_job-' + ts)
else:
    print(analyze_id)

{'status': 'Running', 'starttime': '2019-04-17T22:26:21.4631', 'endtime': None, 'percent_complete': 0, 'job_type': 'AnalyzePredictions', 'loss': None, 'generations': 0, 'dataset_names': None, 'artifact_names': ['Analyze_prediction_artifact-20190417222338'], 'model_name': 'model-20190417222338', 'job_error': ''}
{'status': 'Failed', 'starttime': '2019-04-17T22:26:21.4631', 'endtime': '2019-04-17T22:26:26.115504', 'percent_complete': 100, 'job_type': 'AnalyzePredictions', 'loss': None, 'generations': 0, 'dataset_names': None, 'artifact_names': ['Analyze_prediction_artifact-20190417222338'], 'model_name': 'model-20190417222338', 'job_error': 'TooManyRowsToExplain: Too many rows in dataset to explain. Number of rows is 594, maximum is 500'}


In [22]:
ds.lookup_job_status_name('Analyze_prediction_job-' + ts)

(True,
 {'status': 'Failed',
  'starttime': '2019-04-17T22:26:21.4631',
  'endtime': '2019-04-17T22:26:26.115504',
  'percent_complete': 100,
  'job_type': 'AnalyzePredictions',
  'loss': None,
  'generations': 0,
  'dataset_names': None,
  'artifact_names': ['Analyze_prediction_artifact-20190417222338'],
  'model_name': 'model-20190417222338',
  'job_error': 'TooManyRowsToExplain: Too many rows in dataset to explain. Number of rows is 594, maximum is 500'})

Download and print the top 10 features

In [23]:
status, feature_importance = ds.download_artifact('Analyze_prediction_artifact-' + ts)
feature_importance.head()

AttributeError: 'str' object has no attribute 'head'